# 📦 Step 1: Create Delta Table for Centralized Config Store

This notebook sets up a Delta table (`aerodemo_config_store`) to hold key-value configuration data 
for the AeroDemo project. 

✅ This will replace the need to hardcode or edit Python files to track things like workflow job IDs,  
DLT pipeline IDs, cluster settings, and environment-specific toggles.

We'll create the table once under the shared catalog and schema, and then manage all dynamic configs 
there from other notebooks and workflows.

### What this notebook does:

1. Defines the **catalog** and **schema** for the table location.
2. Defines a Delta table `aerodemo_config_store` with columns:
   - `env`: environment name (e.g., dev, staging, prod)
   - `config_key`: name of the config (e.g., e2e_workflow_job_id)
   - `config_value`: value of the config
3. Overwrites any existing table with a clean schema.
4. Optionally inserts initial config rows for testing.

In [0]:
# Import Spark session
spark = spark

# Set catalog and schema (match your dev environment)
CATALOG = "arao"
SCHEMA = "aerodemo"

TABLE_NAME = f"{CATALOG}.{SCHEMA}.aerodemo_config_store"

print(f"✅ Creating table: {TABLE_NAME}")

# Drop if exists
spark.sql(f"DROP TABLE IF EXISTS {TABLE_NAME}")

# Create new Delta table
spark.sql(f"""
    CREATE TABLE {TABLE_NAME} (
        env STRING,
        config_key STRING,
        config_value STRING
    )
    USING DELTA
    COMMENT 'Key-Value configuration store for AeroDemo project'
""")

print(f"✅ Table {TABLE_NAME} created successfully!")

# Optional: Insert sample configs
sample_data = [
    ("dev", "e2e_workflow_job_id", "123456789"),
    ("dev", "dlt_pipeline_id", "dev-dlt-id"),
    ("dev", "existing_cluster_id", "dev-cluster-id")
]

df = spark.createDataFrame(sample_data, ["env", "config_key", "config_value"])
df.write.format("delta").mode("append").saveAsTable(TABLE_NAME)

print(f"✅ Sample data inserted into {TABLE_NAME}")